# Generate Particle ID
This notebook is used to select a particle group (particle ID) from the estimates generated by the estimate phase of Spyral. If you haven't already run the estimate phase and try to use this notebook, it will fail! You should also be sure to set some configuration values ahead of running this notebook. In particular, the particle_id_file field should be set to a file name that you want to use to save a new particle ID cut!

First lets load our modules:

In [ ]:
import sys
sys.path.append('..')
from spyral_utils.plot import CutHandler, serialize_cut, Histogrammer
from spyral_utils.nuclear import NuclearDataMap
from spyral.core.workspace import Workspace
from spyral.core.config import load_config
from spyral.core.constants import DEG2RAD
from spyral.core.particle_id import load_particle_id

import polars as pl
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from json import load, dump
from ipywidgets.widgets import VBox

RAD2DEG = 1.0/DEG2RAD

With our modules loaded we can now begin loading the current Spyral configuration and workspace.

In [ ]:
# Change the config file name to match your setup
config = load_config(Path("../local_config.json"))
ws = Workspace(config.workspace)
nuclear_map = NuclearDataMap()

Here we'll set some parameters we'll use for the particle ID. You'll want to change these for your specific particle ID.

In [ ]:
# Set these!
pid_name = "proton_cut" # name given to PID
pid_z = 1 # atomic number
pid_a = 1 # mass number
pid_path = ws.get_gate_file_path(config.solver.particle_id_filename) # Path to which we will write our PID


Now we'll create some utility objects from spyral-utils that will help us with the plotting

In [ ]:
grammer = Histogrammer()
handler = CutHandler()

Now we'll add some histograms to the Histogrammer

In [ ]:
grammer.add_hist2d("particle_id", (500, 300), ((-100.0, 5000.0), (0.0, 3.0))) # Plot of dEdx vs. Brho (particle ID)
grammer.add_hist1d("ion_chamber", 4096, (0.0, 4096.0)) # Plot of ion chamber (beam ID)
grammer.add_hist2d("kinematics", (360, 200), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics)

With our histograms created, we're now ready to load the data and fill the histograms!

In [ ]:
for run in range(config.run.run_min, config.run.run_max+1):
    run_path = ws.get_estimate_file_path_parquet(run)
    if not run_path.exists():
        continue
    df = pl.read_parquet(run_path)
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    df = df.filter((pl.col('ic_amplitude') > config.solver.ic_min_val) & (pl.col('ic_amplitude') < config.solver.ic_max_val))
    grammer.fill_hist2d('particle_id', df.select('dEdx').to_numpy(), df.select('brho').to_numpy())
    grammer.fill_hist2d('kinematics', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())
    grammer.fill_hist1d('ion_chamber', df.unique(subset=['event']).select('ic_amplitude').to_numpy())

With our data loaded and histograms filled, we are ready to plot! We'll use the plotly library to make some interactive histograms. First, the ion chamber

In [ ]:
fig = go.Figure()
ic = grammer.get_hist1d("ion_chamber")
fig.add_trace(
    go.Bar(x=ic.bins, y=ic.counts, name="IonChamber", width=ic.bin_width, marker={"color": "rgba(24,69,59,1.0)", "line": {"color": "rgba(24,69,59,1.0)"}})
)
fig.update_layout(
    bargap = 0
)
fig.show()

You should see a plot of the ion chamber above!

Now lets plot the interesting bit, the particle ID. We also need to bind our cut handler to the figure we make, so the code below is a tad more involved.

In [ ]:
fig_wid = go.FigureWidget()
pid_hist = grammer.get_hist2d("particle_id")
fig_wid.add_trace(
    go.Heatmap(x=pid_hist.x_bins, y=pid_hist.y_bins, z=np.log(pid_hist.counts), name="ParticleID")
)
fig_wid.add_trace(
    go.Scatter(x=[1.0], y=[1.0], mode="markers", marker={"size": 3, "opacity": 0.0}, name="Grid")
)
scatter = fig_wid.data[0]
fig_wid.update_xaxes(
    range=[-10,5000]
)
fig_wid.update_yaxes(
    range=[-0.01, 3.0]
)
fig_wid.update_layout(
    xaxis_title="dEdx",
    yaxis_title="B&#961;",
    width=1500,
    height=1000,
    newselection= {
        "line": {
            "color": "white"
        },
    }
)
scatter.on_selection(handler.plotly_on_select)
VBox(
    (fig_wid,)
)


You should see the particle ID spectrum above! Now you can use the plotly LassoSelect or BoxSelect tools to make a gate on a group in the spectrum!

Lets also plot our kinematics

In [ ]:
fig = go.Figure()
kine = grammer.get_hist2d("kinematics")
fig.add_trace(
    go.Heatmap(x=kine.x_bins, y=kine.y_bins, z=np.log(kine.counts), name="Kinematics")
)
fig.update_layout(
    width=1500,
    height=1000
)
fig.show()

You should see a spectrum of the kinematics above!

Once you've done all of that and have *made* a selection in the particle ID, we want to save the particle ID to our workspace. Only run the cell below once you've made a selection!

In [ ]:
print(handler.cuts)

In [ ]:
# If you've made multiple cuts, you'll want to change the name used here. Cuts are automatically named in the order they were made (first cut is cut_0, second cut_1, etc.)
cut = handler.cuts["cut_0"]
cut.name = pid_name
serialize_cut(cut, pid_path)
cut_json = None
# Add the Z, A fields for particle ID
with open(pid_path, "r") as pid_file:
    cut_json =  load(pid_file)
    cut_json["Z"] = pid_z
    cut_json["A"] = pid_a
with open(pid_path, "w") as pid_file:
    dump(cut_json, pid_file)


Great! Now if you look in the PID file, you should see some JSON describing the cut!

As a test, we can now load the particle ID and apply it to itself and our kinematics. This helps us make sure that everything is working correctly. Again, only run this cell if you've already saved a particle ID!

In [ ]:
pid_cut = load_particle_id(pid_path, nuclear_map)
if pid_cut is None:
    raise ValueError("Load particle ID failed!")

grammer.add_hist2d("particle_id_gated", (500, 300), ((-100.0, 5000.0), (0.0, 3.0))) # Plot of dEdx vs. Brho (particle ID), gated on PID
grammer.add_hist2d("kinematics_gated", (360, 200), ((0.0, 180.0), (0.0, 3.0))) # Plot of polar angle vs. Brho (kinematics), gated on PID

for run in range(config.run.run_min, config.run.run_max+1):
    run_path = ws.get_estimate_file_path_parquet(run)
    if not run_path.exists():
        continue
    df = pl.read_parquet(run_path)
    # The below filter is optional. Filter the data on the ion chamber gate. Comment/Uncomment the line below to turn on/off the filter
    # df = df.filter((pl.col('ic_amplitude') > config.solver.ic_min_val) & (pl.col('ic_amplitude') < config.solver.ic_max_val))
    df = df.filter(pl.struct(['dEdx', 'brho']).map_batches(pid_cut.cut.is_cols_inside))
    grammer.fill_hist2d('particle_id_gated', df.select('dEdx').to_numpy(), df.select('brho').to_numpy())
    grammer.fill_hist2d('kinematics_gated', df.select('polar').to_numpy() * RAD2DEG, df.select('brho').to_numpy())

In [ ]:
fig = go.Figure()
pid_gated = grammer.get_hist2d("particle_id_gated")
fig.add_trace(
    go.Heatmap(x=pid_gated.x_bins, y=pid_gated.y_bins, z=pid_gated.counts, name="Particle ID Gated")
)
fig.update_layout(
    width=1500,
    height=1000
)
fig.show()

In [ ]:
fig = go.Figure()
kine_gated = grammer.get_hist2d("kinematics_gated")
fig.add_trace(
    go.Heatmap(x=kine_gated.x_bins, y=kine_gated.y_bins, z=kine_gated.counts, name="Particle ID Gated")
)
fig.update_layout(
    width=1500,
    height=1000
)
fig.show()

You should see the plots above with your cut applied! You now have a particle ID ready for use in the final phase of Spyral!